# Analysis Summary

Our goal from this analysis will be to predict the outcome of March Madness games. To do this, we will be using a random forest classifer that takes in historical data from NCAA tourney matchups as input and outputs predictions for the winning team for each game in an input set of March Madness games. 

### Baseline Model
The baseline model we will use to compare our model's results against will be solely tied to a team's RPI. Our baseline model poses the following hypothesis:

    That in any NCAA tournament game, the team with the lower RPI will win the game.
  
Intuitively, this is a reasonable prediction. RPI (Ratings Percentage Index), ranks teams based on their wins, losses, and strength of schedule for the past season. If team A has a lower RPI than team B at the end of a season, it's generally considered that team A's performance throughout the season has been at a higher level than team B's. For this reason, if we knew nothing else about the two teams, predicting the outcome based on RPI is a good starting point. 
### Random Forest Approach
We would like to build on our baseline model and see if we can develop an approach that more accurately predicts the outcome of games. One of the most exciting parts of March Madness is the array of upsets that occur throughout the tournament. In general, upsets occur when a team with a higher RPI beats a team with a lower RPI. We would like to create a model that performs better than our baseline model by more accurately predicting the outcome of games, in particular predicting when upsets occur. For us to accomplish this, we can utilize RPI as well as some additional attributes that provide more information surrounding each team's level of performance in the past season. These factors can be used to build a random forest in order to: 
    1. Predict when an upset is going to occur in March Madness
    2. Identify which factors are correlated to predicting the outcome of a game

We will build a random forest by passing as input a data frame where each row corresponds to an NCAA tournament game. Each row will contain data regarding each team's yearly averages and totals in statistical categories, RPI, the game's outcome, and whether or not the team with the lowest RPI won. This last piece of information will be our dependent variable. The random forest will utilize features in our training data describing the winning team and losing team's performance during the season, in order to learn which factors are tied to predicting the outcome of a game. Once the random forest has been trained, tournament data where the outcome of the each game has been excluded can be used as input to the model to generate a set of predictions for each game in that year's tournament and test the accuracy of our random forest classifer.

### Data Sources Overview
The data the was used to perform this analysis came from the Google Cloud & NCAA® ML Competition 2018-Men's Kaggle competition (Insert Link). The following is a brief summary of all the datasets that were used:
- NCAATourneyCompactResults: Contains records from each NCAA tournament game from 1985-2017, including score and region.  
- Teams: Contains information for each Division 1 (D1) basketball team including an ID, name, first and most recent year playing D1 basketball. 
- MasseyOrdinals_Prelim2018: Contains data from 2003-2017 surrounding each D1 team's rank from various rankings sources throughout. 
- RegularSeasonDetailedResults: Contains similar information as the NCAATourneyCompactResults dataset, with the addition that each row will also contain the totals in a variety of statistical categories for the winning team and losing team. These are categories that are often found in a boxscore. 

A more in depth description of each of the datasets that were used and additional datasets provided by Kaggle can be found at https://www.kaggle.com/c/mens-machine-learning-competition-2018/data


### Data Cleansing & Preparation

In order to create our random forest, we need to complete the following:
    1. Settle on a set of statistics to evaluate the performance of each team. 
    2. Create a data frame that represents a matchup that occurred in an NCAA tournament and contains our set of statistics for each team in the matchup 

To accomplish the first task, the following statistics were calculated for each team:
    - Defensive Rebounds (DRB)
    - Efficient Field Goal Percentage (EFG)
    - Free Throws Attempted (FTA)
    - Free Throw Percentage (FTP)
    - Margin of Loss (MOL)
    - Margin of Victory (MOV)
    - Offensive Rebounds (ORB)
    - Possessions (POSS)
    - Turnovers (TO)
    - Turnovers Forced (TOF)
    - Defensive Efficiency (dEff)
    - Offensive Effiency (oEff)
    - Rankings Percentage Index (RPI)
    - Conference Tourney Wins (confTournWins)
    - Wins vs. a Tournament Team (winsVsTourney)
    - Number of Games Played (numGamesPlayed)


With the exception of the last four, the yearly pre-tournament averages (regular season and conference tourney play) were found. The last three are cumulative totals, and RPI was found by using the latest ranking given to a team before the tournament. 

By parsing through the 4 datasets listed above, we can calculate the desired information for each team in a March Madness tournament from 2003-2017 as seen below:

In [5]:
matchups = getMatchupData()
matchups.head()

,dayNum,lDRB,lEFG,lFTA,lFTP,lMOL,lMOV,lORB,lPOSS,lRPI,...,wRPI,wTO,wTOF,w_id,wconfTournWins,wdEff,wname,wnumGamesPlayed,woEff,wwinsVsTourney
0,134,24.800000,0.594340,28.066667,0.613745,9.658182,7.331579,13.166667,68.560000,217,...,186,16.206897,12.827586,1421_2003,2,111.784408,UNC Asheville,29,101.660568,0
1,136,25.724138,0.532609,19.551724,0.649708,6.891836,11.673356,12.965517,64.786207,144,...,2,14.785714,16.857143,1112_2003,1,92.061625,Arizona,28,111.410521,10
2,136,25.965517,0.539216,22.896552,0.628299,3.711207,12.428413,14.068966,68.882759,26,...,32,14.000000,15.517241,1113_2003,1,104.177110,Arizona St,29,114.520186,4
3,136,23.181818,0.591667,20.030303,0.689707,1.078704,18.182033,10.878788,67.951515,23,...,44,18.241379,16.068966,1141_2003,4,103.380167,C Michigan,29,111.938246,0
4,136,22.033333,0.533898,20.466667,0.778148,6.902834,17.403517,9.733333,65.986667,53,...,33,14.172414,14.931034,1143_2003,1,97.814892,California,29,104.381997,4


Each row in the output data frame represents a game played in an March Madness tournament. The 16 statistics that were targeted are used features as well as some additional information to help identify which teams played and when the game occurred. A more detailed look at how the Matchups data frame was created can be seen in the getMatchupData function in the Appendix. 

### Random Forest Creation

The matchups data frame can be used as input to a random forest classifier to output predictions for a subset of NCAA tournament games. When creating a random forest classifer, a few parameters should be considered:

- _Maximum Number of Features_: The maximum number of features that our algorithm uses to create a decision tree that will be a part of our random forest. A value of sqrt(n) where n is the number of features in our input dataset was chosen. This is generally considered to be a good starting point for random forest classifiers because it allows the decision trees that are created to have a strong chance of selecting a unique set of features to train a model with. The more unique our set of decision trees are, the more confident we can be in our random forest's performance on test data. 
- _Number of Trees_: The number of trees in a random forest impacts the classifier's effectiveness for similar reason. If not enough decision trees are chosen, some features may not be included in our model and the classifer's effectiveness is hindered. The more trees in a forest, the more likely we cover the full feature space. To have a strong chance of covering the full feature space, 1000 trees were selected to be in our random forest.

For this project, random forests classifiers were created for two use cases:
    1. Predicting the outcome of all NCAA tournament games from one season between 2003-2017
    2. Predicting the outcome of all NCAA tournament championship games from 2003-2017

In order to accomplish both tasks, two methods were created, getPredictionsChips and getPredictions. Both methods behave similarly, except getPredictionChips uses championship games as its test data set to output predictions and getPredictions uses a specified year as its test data set. In each method, a data set of historical NCAA tournament games is cleaned up by removing any qualitative columns or any column specifying a result from our baseline model. Once the training and test data sets are cleaned, a random forest classifier is created using the training data and predictions are made on the test data using the generated classifier. Each method returns a tuple with the following information: 
- _Output Predictions_ (outputPreds): A dataframe describing the prediction for each matchup, the actual result, and the probability for the predicted outcome to occur
- _Baseline Model Accuracy_ (baselineAcc): The accuracy from our baseline model that predicted a game's outcome based on which team had the lower RPI
- _Random Forest Classifier Accuracy_ (modelAcc): The accuracy from our random forest classifier that predicted a game's outcome on a variety of input features

### TODO: Random Forest Classifier Results vs. Baseline Model
- 15 Year comparison with baseline model
- Championship game predictions vs. baseline model

- Two sets of results to look at
1. Results of chip game analysis
2. Results of per season analysis 

For each set of reslts, we compared the classifier's prediction accuracy to the baseline model's. In all years of games analyzed,our model always outperformed the baseline model. This tells us that we can be smarter when picking games and not simply rely on RPI.

In [416]:
indPredicts = []
outCols = ["Predict", "Actual", "A Name", "A ID", "B Name", "B ID", "Prob For", "Prob Against"]
baseAccs = []
modelAccs = []

# Chip games testing
outputPreds, baselineAcc, modelAcc = getPredictionsChips()
print("Baseline Model Accuracy: {}".format(round(baselineAcc, 2)))
print("Our Model's Accuracy: {}".format(round(modelAcc, 2)))

for row in outputPreds:
    indPredicts.append(row.tolist())
predsDF = pd.DataFrame(indPredicts, columns = outCols)
# predsDF
# pd.DataFrame(indPredicts).to_csv("data/output/chipTestResults.csv", index=False, header=True)
# outputPreds


[1 1 1 1 0 1 1 1 1 1 1 1 1 1 1]
[0 1 0 0 0 0 1 1 1 1 1 0 0 1 1]
-21.0
15
Baseline Model Accuracy: 0.53
Our Model's Accuracy: 0.6


In [119]:
indPredicts = []
outCols = ["Predict", "Actual", "L Name", "L ID", "W Name", "W ID", "Prob For", "Prob Against"]
baseAccs = []
modelAccs = []
for i in range(2003, 2018):
#     outputPreds, baselineAcc, modelAcc = getPredictions(str(i))
    outputPreds, baselineAcc, modelAcc = getPredictionsDeltas(str(i))
    baseAccs.append(round(baselineAcc, 2))
    modelAccs.append(round(modelAcc, 2))
    for row in outputPreds:
        indPredicts.append(row.tolist())

pd.DataFrame(indPredicts).to_csv("data/output/results_deltas_A.csv", index=False, header=True)
accDF = pd.DataFrame({"Season": range(2003,2018), "Baseline": baseAccs, "RF": modelAccs})
predsDF = pd.DataFrame(indPredicts, columns = outCols)

                 importance
RPI                0.143598
dEff               0.117088
oEff               0.113550
FTP                0.106366
ORB                0.102012
FTA                0.096640
DRB                0.093948
EFG                0.093527
winsVsTourney      0.086522
confTourneyWins    0.046749
                 importance
RPI                0.147919
dEff               0.116424
oEff               0.113024
FTP                0.107955
ORB                0.103806
FTA                0.095640
DRB                0.094743
EFG                0.089856
winsVsTourney      0.085956
confTourneyWins    0.044678
                 importance
RPI                0.152091
oEff               0.113004
dEff               0.111858
FTP                0.105411
ORB                0.105019
FTA                0.094732
DRB                0.092904
EFG                0.091769
winsVsTourney      0.086227
confTourneyWins    0.046986
                 importance
RPI                0.144117
dEff               0

In [ ]:
predsDF.head()

In [118]:
accDF

,Season,Baseline,RF
0,2003,0.70,0.72
1,2004,0.73,0.72
2,2005,0.73,0.75
3,2006,0.64,0.62
4,2007,0.73,0.73
5,2008,0.73,0.72
6,2009,0.73,0.70
7,2010,0.69,0.77
8,2011,0.63,0.60
9,2012,0.66,0.76


In [114]:
import pandas as pd 
import numpy as np
import team, game as g
from sklearn.ensemble import RandomForestClassifier
# Used for developing visual of Random Forest if desired
from sklearn.tree import export_graphviz
from sklearn.externals.six import StringIO
from IPython.display import Image
import pydotplus

def getTeamNames():
    """
    Return dictionary where key is team ID and value is team name
    """
    names = {}
    teams = pd.read_csv("Data/Teams.csv")
    for index, row in teams.iterrows():
        teamId = row["TeamID"]
        name = row["TeamName"]
        names[teamId] = name
    return names

def getSeasonStats(ncaaTourneyTeams):
    """
    Use regular season results and RPI rankings to create a 
    dictionary where key is the team's ID and the value is a 
    Team object. Team objects contain yearly avg stats for each 
    team in various categories.
    """
    teams = {}
    names = getTeamNames()
    unfiltRanks = pd.read_csv("data/MasseyOrdinals_Prelim2018.csv")
    rankings = unfiltRanks[(unfiltRanks["SystemName"] == "RPI") & (unfiltRanks["RankingDayNum"] == 133)]
    regSeasonResults = pd.read_csv("data/RegularSeasonDetailedResults.csv")
    for index, row in regSeasonResults.iterrows():
        season = row["Season"]
        dayNum = row["DayNum"]
        wTeamId = row["WTeamID"]
        lTeamId = row["LTeamID"]
        customWId = str(wTeamId) + "_" + str(season)
        customLId = str(lTeamId) + "_" + str(season)
        wRPI = None
        lRPI = None
        try:
            wRPI = rankings[(rankings["Season"] == season) & (rankings["TeamID"] == wTeamId)].iloc[0]["OrdinalRank"]
            lRPI = rankings[(rankings["Season"] == season) & (rankings["TeamID"] == lTeamId)].iloc[0]["OrdinalRank"]
        except Exception as e:
            pass
            # print str(lTeamId) + " " + str(season) + " not found"
        
        if customWId not in teams:
            teams[customWId] = team.Team(customWId)
        if customLId not in teams:
            teams[customLId] = team.Team(customLId)
        wTeam = teams[customWId]
        wTeam.RPI = wRPI
        wTeam.name = names[wTeamId]
        wTeam.updateStats(row, True)
        if customLId in ncaaTourneyTeams:
            wTeam.winsVsTourney += 1
        lTeam = teams[customLId]
        lTeam.name = names[lTeamId]
        lTeam.RPI = lRPI
        lTeam.updateStats(row, False)
    return teams

def populateNCAATourneyTeams():
    """
    Create an ID for each team using a combination of its id and the season the team played in. 
    Output a dictionary with an entry for each team whose key is its newly created id
    """
    ncaaTourneyTeams = {}
    ncaaTournResults = pd.read_csv("data/NCAATourneyCompactResults.csv")
    for index, row in ncaaTournResults.iterrows():
        season = row["Season"]
        dayNum = row["DayNum"]
        wTeamId = row["WTeamID"]
        lTeamId = row["LTeamID"]
        customWId = str(wTeamId) + "_" + str(season)
        customLId = str(lTeamId) + "_" + str(season)

        if customWId not in ncaaTourneyTeams:
            ncaaTourneyTeams[customWId] = 1
        if customLId not in ncaaTourneyTeams:
            ncaaTourneyTeams[customLId] = 1
    return ncaaTourneyTeams

def getMatchups(teams):
    """
    Use NCAA Tournament results to return data frame of matchups where each row contains data for one matchup between two teams, including their yearly avg totals in statistical categories, RPI, and game result.
    """
    matchups = []
    ncaaTournResults = pd.read_csv("data/NCAATourneyCompactResults.csv")
    for index, row in ncaaTournResults.iterrows():
        season = row["Season"]
        dayNum = row["DayNum"]
        wTeamId = row["WTeamID"]
        lTeamId = row["LTeamID"]
        customWId = str(wTeamId) + "_" + str(season)
        customLId = str(lTeamId) + "_" + str(season)

        if customWId in teams and customLId in teams:
            wTeamData = teams[customWId].objToDict().copy()
            for key in wTeamData.keys():
                if key[0] == "w":
                    wTeamData[key] = wTeamData[key]
                else:
                    wTeamData["w" + key] = wTeamData[key]
                del wTeamData[key]
            lTeamData = teams[customLId].objToDict().copy()
            for key in lTeamData.keys():
                if key[0] == "l":
                    lTeamData[key] = lTeamData[key]
                else:
                    lTeamData["l" + key] = lTeamData[key]
                del lTeamData[key]
            matchupData = wTeamData.copy()
            matchupData.update(lTeamData)
            matchupData["dayNum"] = dayNum
            matchupData["season"] = season
            matchups.append(matchupData)
    colOrder = ["dayNum", "season", "l_id", "lname", "w_id", "wname", "lDRB", "lEFG", "lFTA", "lFTP", "lMOL", "lMOV", "lORB", "lPOSS",
                "lRPI", "lTO", "lTOF", "lconfTournWins", "ldEff", "lnumGamesPlayed", "loEff", "lwinsVsTourney",
                "wDRB", "wEFG", "wFTA", "wFTP", "wMOL", "wMOV", "wORB", "wPOSS", "wRPI", "wTO", "wTOF", 
                "wconfTournWins", "wdEff", "wnumGamesPlayed", "woEff", "wwinsVsTourney"]
    df = pd.DataFrame.from_dict(matchups)
    print(df)
    print(df.columns)
    print(len(df.columns))
    df = df[colOrder]
    return df

def getMatchupData():
    """
    Returns data frame of historical matchups in NCAA tournament.
    Reads in existing CSV if available. Otherwise, produces data frame by creating Team objects, calculating yearly avg totals for each team, and joining with historical NCAA tourney matchup data
    """
    try:
        matchups = pd.read_csv("Data/output/matchups.csv")
        return matchups
    except Exception as e:
        ncaaTourneyTeams = populateNCAATourneyTeams()
        teamObjs = getSeasonStats(ncaaTourneyTeams)
        matchups = getMatchups(teamObjs)
        matchups.to_csv("Data/output/matchups.csv", index=False)
        return matchups
    
def findChampionshipMatches():
    """
    Read in NCAA tourney matchups and return data frame containing additional column denoting (True/False) if that matchup was a championship game. 
    """
    matchups = getMatchupData()
    matchups = sortMatchups(matchups)
    ## group by season and with resulting groupby obj, find whether each row equals the dayNum max for each group
    ## store result as column in matchups defining whether championship played that day
    ## able to pass in functions to transform to perform calculations for each group
    matchups["chipGame"] = matchups.groupby(['season'])['dayNum'].transform(max) == matchups['dayNum']
    return matchups

def getPredictionsChips():
    """
    Outputs predictions for all championship games from 2003-2017 using a Random Forest classifier. Baseline model takes team with lower RPI as winner. 
    Returns a tuple consisting of a data frame containing the model's prediction for every matchup in our test dataset, the baseline model's accuracy, our model's accuracy
    """
    matchups = findChampionshipMatches()
    cols = list(matchups.columns)
    
    # Create training/test data sets
    train = matchups[matchups["chipGame"] == False]
    test = matchups[matchups["chipGame"] == True]
    baselineAcc = 1.0*sum(test["baseline"]) / test.shape[0]
    
    trainLabels = np.array(train["baseline"])
    trainLabels = trainLabels.astype(int)
    testLabels = np.array(test["baseline"])
    testLabels = testLabels.astype(int)
    testNames = np.column_stack((test["aname"], test["a_id"], test["bname"], test["b_id"]))
    # Drop qualitative & output columns
    train = train.drop(["b_id", "a_id", "baseline", "bname", "aname", "season", "dayNum", "chipGame"], axis = 1)
    test = test.drop(["b_id", "a_id", "baseline", "bname", "aname", "season", "dayNum", "chipGame"], axis = 1)
    feature_names = train.columns
    trainFeatures = np.array(train)
    testFeatures = np.array(test)
    maxFeatures = int(len(feature_names)**0.5)

    rf = RandomForestClassifier(n_estimators = 1000, random_state=42, oob_score=True, max_features=maxFeatures)
    rf.fit(trainFeatures, trainLabels)
    ## Draw sample classification tree
    # drawTree(rf, "sampleTree")

    predictions = rf.predict(testFeatures)
    predictProbs = rf.predict_proba(testFeatures)
    modelAcc = 1.0*(predictions.shape[0] - sum(predictions ^ testLabels)) / predictions.shape[0]
    stack = np.column_stack((predictions.T, testLabels.T, testNames[:,0], testNames[:,1], testNames[:,2], testNames[:,3], predictProbs[:,0], predictProbs[:,1]))
    outputPreds = stack[stack[:,0].argsort()]
    return outputPreds, baselineAcc, modelAcc

### Utilize historical matchup data to build RF model. 
def getPredictions(year, train=None, test=None):
    """
    Outputs predictions for games from test data set using a Random Forest classifier. Baseline model takes team with lower RPI as winner. 
    Returns a tuple consisting of a data frame containing the model's prediction for every matchup in our test dataset, the baseline model's accuracy, our model's accuracy
    """
    matchups = getMatchupData()
    matchups = sortMatchups(matchups)
    cols = list(matchups.columns)
    train = matchups[~matchups["b_id"].str.contains(year)]
    test = matchups[matchups["b_id"].str.contains(year)]
    baselineAcc = 1.0*sum(test["baseline"]) / test.shape[0]
    
    # Create training/test data sets
    trainLabels = np.array(train["baseline"])
    trainLabels = trainLabels.astype(int)
    testLabels = np.array(test["baseline"])
    testLabels = testLabels.astype(int)
    testNames = np.column_stack((test["aname"], test["a_id"], test["bname"], test["b_id"]))
    # Drop qualitative & output columns
    train = train.drop(["b_id", "a_id", "baseline", "bname", "aname", "season", "dayNum"], axis = 1)
    test = test.drop(["b_id", "a_id", "baseline", "bname", "aname", "season", "dayNum"], axis = 1)
    
#     aDRB	aEFG	aFTA	aFTP	aMOL	aMOV	aORB	aPOSS	aRPI	aTO	aTOF	a_id	
#     aconfTournWins	adEff	aname	anumGamesPlayed	aoEff	awinsVsTourney	season
    
    train = train.drop(["aMOV", "aMOL", "aFTA", "aFTP", "anumGamesPlayed", "aRPI", "bMOV", "bMOL", "bFTA", "bFTP", "bnumGamesPlayed", "bRPI"], axis = 1)
    test = test.drop(["aMOV", "aMOL", "aFTA", "aFTP", "anumGamesPlayed", "aRPI", "bMOV", "bMOL", "bFTA", "bFTP", "bnumGamesPlayed", "bRPI"], axis = 1)
    feature_names = train.columns
    trainFeatures = np.array(train)
    testFeatures = np.array(test)
    maxFeatures = int(len(feature_names)**0.5)

    rf = RandomForestClassifier(n_estimators = 1000, random_state=42, oob_score=True, max_features=maxFeatures)
    rf.fit(trainFeatures, trainLabels)
    
    feature_importances = pd.DataFrame(rf.feature_importances_,
                                       index = train.columns,
                                       columns=['importance']).sort_values('importance', ascending=False)
    
    print(feature_importances)

    predictions = rf.predict(testFeatures)
    predictProbs = rf.predict_proba(testFeatures)
    modelAcc = 1.0*(predictions.shape[0] - sum(predictions ^ testLabels)) / predictions.shape[0]
    stack = np.column_stack((predictions.T, testLabels.T, testNames[:,0], testNames[:,1], testNames[:,2], testNames[:,3], predictProbs[:,0], predictProbs[:,1]))
    outputPreds = stack[stack[:,0].argsort()]
    return outputPreds, baselineAcc, modelAcc

### Utilize historical matchup data to build RF model. 
def getPredictionsDeltas(year, train=None, test=None):
    """
    Outputs predictions for games from test data set using a Random Forest classifier. Baseline model takes team with lower RPI as winner. 
    Returns a tuple consisting of a data frame containing the model's prediction for every matchup in our test dataset, the baseline model's accuracy, our model's accuracy
    """
    matchups = getMatchupData()
    matchups = sortMatchups(matchups)
    deltas = pd.DataFrame(findAllDeltas(sortedMatch))
    cols = list(deltas.columns)
    train = deltas[~deltas["b_id"].str.contains(year)]
    test = deltas[deltas["b_id"].str.contains(year)]
    baselineAcc = 1.0*sum(test["baseline"]) / test.shape[0]
    
    # Create training/test data sets
    trainLabels = np.array(train["baseline"])
    trainLabels = trainLabels.astype(int)
    testLabels = np.array(test["baseline"])
    testLabels = testLabels.astype(int)
    testNames = np.column_stack((test["aname"], test["a_id"], test["bname"], test["b_id"]))
    # Drop qualitative & output columns
    train = train.drop(["b_id", "a_id", "baseline", "bname", "aname", "season", "dayNum"], axis = 1)
    test = test.drop(["b_id", "a_id", "baseline", "bname", "aname", "season", "dayNum"], axis = 1)
    
#     aDRB	aEFG	aFTA	aFTP	aMOL	aMOV	aORB	aPOSS	aRPI	aTO	aTOF	a_id	
#     aconfTournWins	adEff	aname	anumGamesPlayed	aoEff	awinsVsTourney	season

    train = train.drop(["MOL", "MOV", "POSS", "TO", "TOF", "numGamesPlayed"], axis = 1)
    test = test.drop(["MOL", "MOV", "POSS", "TO", "TOF", "numGamesPlayed"], axis = 1)
    
    feature_names = train.columns
    trainFeatures = np.array(train)
    testFeatures = np.array(test)
    maxFeatures = int(len(feature_names)**0.5)

    rf = RandomForestClassifier(n_estimators = 1000, random_state=42, oob_score=True, max_features=maxFeatures)
    rf.fit(trainFeatures, trainLabels)
    
    feature_importances = pd.DataFrame(rf.feature_importances_,
                                       index = train.columns,
                                       columns=['importance']).sort_values('importance', ascending=False)
    
    print(feature_importances)

    predictions = rf.predict(testFeatures)
    predictProbs = rf.predict_proba(testFeatures)
    modelAcc = 1.0*(predictions.shape[0] - sum(predictions ^ testLabels)) / predictions.shape[0]
    stack = np.column_stack((predictions.T, testLabels.T, testNames[:,0], testNames[:,1], testNames[:,2], testNames[:,3], predictProbs[:,0], predictProbs[:,1]))
    outputPreds = stack[stack[:,0].argsort()]
    return outputPreds, baselineAcc, modelAcc

def drawTree(rf, treeName):
    dot_data = StringIO()
    export_graphviz(rf.estimators_[0], out_file=dot_data, filled=True, rounded=True, special_characters=True, feature_names=feature_names)
    graph = pydotplus.graph_from_dot_data(dot_data.getvalue())
    graph.write_pdf("{}.pdf".format(treeName))
    
    
def sortRowByRPI(matchup):
    if matchup["lRPI"] > matchup["wRPI"]:
        numCols = matchup.shape[0]
        newOrder = [0, 1, 4, 5, 2, 3] + list(range(22, numCols - 1)) + list(range(6,22)) + [numCols - 1]
        matchup = matchup[matchup.index[newOrder]]
        return list(matchup.values)
    return list(matchup.values)

def findDeltasForMatch(matchup):
    numCols = matchup.shape[0]
    return list(matchup.iloc[0:6].values) + list(matchup.iloc[6:22].values - matchup.iloc[22:numCols - 1].values) + [int(matchup.iloc[numCols - 1])]
        

In [111]:
def sortMatchups(matchups):
    matchups["baseline"] = matchups["wRPI"] < matchups["lRPI"]
    matchups["baseline"].replace(False, 0, inplace=True)
    matchups["baseline"].replace(True, 1, inplace=True)
    sortedMatchups = matchups.apply(sortRowByRPI, axis = 1, result_type = "broadcast")

    newColNames = []
    columns = sortedMatchups.columns
    for name in columns:
        if name[0] == 'l':
            newColNames.append("a" + name[1:])
        elif name[0] == "w":
            newColNames.append("b" + name[1:])
        else:
            newColNames.append(name)
    sortedMatchups.columns = newColNames
    return sortedMatchups

def findAllDeltas(matchups):
    # expand results to extract each item from list returned each iteration of apply function
    deltas = matchups.apply(findDeltasForMatch, axis = 1, result_type = "expand")
    colNames = list(matchups.columns[0:6]) + ["DRB", "EFG", "FTA", "FTP", "MOL", "MOV", "ORB", "POSS", "RPI",
                                             "TO", "TOF", "confTourneyWins", "dEff", "numGamesPlayed", "oEff",
                                             "winsVsTourney", "baseline"]
    deltas.columns = colNames
    return deltas

#TODO: look at getPrediction methods and update column filtering 

In [135]:
matchups = getMatchupData()
sortedMatch = sortMatchups(matchups)
deltas = pd.DataFrame(findAllDeltas(sortedMatch))
# pd.DataFrame(deltas).to_csv("data/output/matchupDeltas.csv", index=False, header=True)
# deltas

,dayNum,season,a_id,aname,b_id,bname,DRB,EFG,FTA,FTP,...,POSS,RPI,TO,TOF,confTourneyWins,dEff,numGamesPlayed,oEff,winsVsTourney,baseline
0,134,2003,1421_2003,UNC Asheville,1411_2003,TX Southern,-1.627586,-0.033995,-7.135632,0.152397,...,0.536552,-31,0.973563,-1.505747,-2,8.881319,-1,-4.024681,0,1
1,136,2003,1112_2003,Arizona,1436_2003,Vermont,1.918719,-0.072381,5.448276,0.051446,...,10.535222,-142,0.716749,3.857143,-2,-5.043444,-1,7.206199,10,1
2,136,2003,1272_2003,Memphis,1113_2003,Arizona St,2.655172,0.024723,-3.310345,-0.047369,...,1.193103,-6,-0.206897,-0.448276,1,-10.216199,0,-8.141029,0,0
3,136,2003,1166_2003,Creighton,1141_2003,C Michigan,-0.094044,-0.110714,-5.142111,-0.073034,...,-2.462278,-21,-4.877743,0.991641,-1,-10.924016,4,1.802002,5,0
4,136,2003,1143_2003,California,1301_2003,NC State,2.345977,-0.007113,-0.949425,-0.089516,...,3.475402,-20,-0.027586,0.297701,-1,-4.884844,-1,-5.197115,2,1
5,136,2003,1140_2003,BYU,1163_2003,Connecticut,-3.480645,-0.017213,2.061290,0.083367,...,-8.279785,-9,-2.058065,1.117204,-1,-0.107735,1,1.630238,0,0
6,136,2003,1181_2003,Duke,1161_2003,Colorado St,-0.366667,-0.057341,3.800000,-0.002037,...,4.686667,-63,-2.100000,5.600000,0,-9.557800,0,4.081426,4,1
7,136,2003,1153_2003,Cincinnati,1211_2003,Gonzaga,-1.929724,-0.055918,-3.109447,-0.032301,...,-4.165438,-14,-3.941244,-0.942396,-1,-2.941761,-3,-6.806612,-2,0
8,136,2003,1228_2003,Illinois,1443_2003,WKU,2.141935,0.054746,-1.996774,0.059099,...,-0.468602,-40,-1.994624,-0.908602,1,-4.571744,-1,6.004937,5,1
9,136,2003,1242_2003,Kansas,1429_2003,Utah St,4.533333,-0.091667,3.900000,-0.045717,...,12.926667,-73,2.666667,4.933333,-2,-12.252048,0,-1.870505,6,1


In [154]:

def mergeRankings(deltas):
    seeds = pd.read_csv("data/NCAATourneySeeds.csv")
    seeds["id"] = seeds.TeamID.astype(str).str.cat(seeds.Season.astype(str), sep='_')
    seeds["Seed"] = seeds.Seed.str.extract('(\d+)', expand=False).astype(int)
    seeds = seeds.drop(["Season", "TeamID"], axis = 1)
    deltas = deltas.merge(seeds, how='left', left_on = "a_id", right_on = "id")
    deltas = deltas.rename(index=str, columns={"Seed": "a_seed"})
    deltas = deltas.merge(seeds, how='left', left_on = "b_id", right_on = "id")
    deltas = deltas.rename(index=str, columns = {"Seed": "b_seed"})
    return deltas

matchups = getMatchupData()
sortedMatch = sortMatchups(matchups)
deltas = pd.DataFrame(findAllDeltas(sortedMatch))
rankedMatches = mergeRankings(deltas)
rankedMatches

,dayNum,season,a_id,aname,b_id,bname,DRB,EFG,FTA,FTP,...,confTourneyWins,dEff,numGamesPlayed,oEff,winsVsTourney,baseline,a_seed,id_x,b_seed,id_y
0,134,2003,1421_2003,UNC Asheville,1411_2003,TX Southern,-1.627586,-0.033995,-7.135632,0.152397,...,-2,8.881319,-1,-4.024681,0,1,16,1421_2003,16,1411_2003
1,136,2003,1112_2003,Arizona,1436_2003,Vermont,1.918719,-0.072381,5.448276,0.051446,...,-2,-5.043444,-1,7.206199,10,1,1,1112_2003,16,1436_2003
2,136,2003,1272_2003,Memphis,1113_2003,Arizona St,2.655172,0.024723,-3.310345,-0.047369,...,1,-10.216199,0,-8.141029,0,0,7,1272_2003,10,1113_2003
3,136,2003,1166_2003,Creighton,1141_2003,C Michigan,-0.094044,-0.110714,-5.142111,-0.073034,...,-1,-10.924016,4,1.802002,5,0,6,1166_2003,11,1141_2003
4,136,2003,1143_2003,California,1301_2003,NC State,2.345977,-0.007113,-0.949425,-0.089516,...,-1,-4.884844,-1,-5.197115,2,1,8,1143_2003,9,1301_2003
5,136,2003,1140_2003,BYU,1163_2003,Connecticut,-3.480645,-0.017213,2.061290,0.083367,...,-1,-0.107735,1,1.630238,0,0,12,1140_2003,5,1163_2003
6,136,2003,1181_2003,Duke,1161_2003,Colorado St,-0.366667,-0.057341,3.800000,-0.002037,...,0,-9.557800,0,4.081426,4,1,3,1181_2003,14,1161_2003
7,136,2003,1153_2003,Cincinnati,1211_2003,Gonzaga,-1.929724,-0.055918,-3.109447,-0.032301,...,-1,-2.941761,-3,-6.806612,-2,0,8,1153_2003,9,1211_2003
8,136,2003,1228_2003,Illinois,1443_2003,WKU,2.141935,0.054746,-1.996774,0.059099,...,1,-4.571744,-1,6.004937,5,1,4,1228_2003,13,1443_2003
9,136,2003,1242_2003,Kansas,1429_2003,Utah St,4.533333,-0.091667,3.900000,-0.045717,...,-2,-12.252048,0,-1.870505,6,1,2,1242_2003,15,1429_2003
